# Using KGTK query to do interesting queries in Wikidata
This notebook shows use cases of interesting queries on Wikidata that can be done using the KGTK query command (aka Kypher), and that cannot be done using the public Wikidata SPARQL endpoint

The notebook has a preamble to set up environment variables to access the relevant files

In [3]:
# Parameters

# Folder on local machine where to create the output and temporary folders
output_path = "/Users/pedroszekely/Downloads/kypher"

# The names of the output and temporary folders
output_folder = "wd-workshop"
temp_folder = "temp.wd-workshop"

# The location of input Wikidata files
wikidata_folder = "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/"
# wikidata_folder = "/Users/pedroszekely/Downloads/kypher/wikidata_os_v1/"
# The wikidata_os files can be downloaded from https://drive.google.com/drive/folders/1V6oAQKmwQ4LJnrBai-uv5gHWphFSCt50?usp=sharing

wikidata_dbpedia_folder = "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-dbpedia"

# Location of the cache database for kypher
cache_path = "/Users/pedroszekely/Downloads/kypher/temp.novartis"
# cache_path = "/Users/pedroszekely/Downloads/kypher/temp.useful_wikidata_files_v4/wikidata.sqlite3.db"
# Whether to delete the cache database
delete_database = False

# shortcuts to commands
kgtk = "time kgtk --debug"
# kgtk = "kgtk --debug"
# kgtk = "kgtk"

In [4]:
import io
import os
import subprocess
import sys
import time

import numpy as np
import pandas as pd

import altair as alt

import papermill as pm

In [5]:
start_time = round(time.time())

In [6]:
# The names of files in the KGTK Wikidata distirbution that we will use in this notebook.
file_names = {
    "claims": "claims.tsv.gz",
    "quantity": "claims.quantity.tsv.gz",
    "time": "claims.time.tsv.gz",
    "label": "labels.en.tsv.gz",
    "alias": "aliases.en.tsv.gz",
    "description": "descriptions.en.tsv.gz",
    "item": "claims.wikibase-item.tsv.gz",
    "external_id": "claims.external-id.tsv.gz",
    "qualifiers": "qualifiers.tsv.gz",
    "sitelinks": "sitelinks.tsv.gz",
    "qualifiers_time": "qualifiers.time.tsv.gz",
    "property_datatypes": "metadata.property.datatypes.tsv.gz",
    "isa": "derived.isa.tsv.gz",
    "p279star": "derived.P279star.tsv.gz",
    "p279": "derived.P279.tsv.gz",
    "p31": "derived.P31.tsv.gz",
    "dwd_isa": "derived.dwd_isa.tsv.gz"
}

# We will define environment variables to hold the full paths to the files as we will use them in the shell commands
kgtk_environment_variables = []

os.environ['WIKIDATA'] = wikidata_folder
kgtk_environment_variables.append('WIKIDATA')

for key, value in file_names.items():
    variable = key.upper()
    os.environ[variable] = wikidata_folder + value
    kgtk_environment_variables.append(variable)

os.environ["WD2DB"] = "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-dbpedia/wikidata_to_dbpedia_edge_file.tsv.gz"
kgtk_environment_variables.append("WD2DB")


# KGTK creates a SQLite database to index the knowledge graph.
if cache_path:
    os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(cache_path)
else:
    os.environ['STORE'] = "{}/{}/wikidata.sqlite3.db".format(output_path, temp_folder)
kgtk_environment_variables.append('STORE')

# We will create many temporary files, so set up a folder for outputs and one for the temporary files.
os.environ['TEMP'] = "{}/{}".format(output_path, temp_folder) 
os.environ['OUT'] = "{}/{}".format(output_path, output_folder) 
kgtk_environment_variables.append('TEMP')
kgtk_environment_variables.append('OUT')

# Envronment variables with shortcuts to the commands we use often
os.environ['kgtk'] = kgtk
# Use for debugging, but careful as it causes import to dataframes to break
os.environ['kypher'] = "time kgtk --debug query --graph-cache " + os.environ['STORE']
# os.environ['kypher'] = "kgtk query --graph-cache " + os.environ['STORE']
kgtk_environment_variables.append('kgtk')
kgtk_environment_variables.append('kypher')

# We'll save the current working directory so we can call into other example notebooks later
os.environ["EXAMPLES_DIR"] = os.getcwd()
kgtk_environment_variables.append('EXAMPLES_DIR')

kgtk_environment_variables.sort()
for variable in kgtk_environment_variables:
    print("{}: \"{}\"".format(variable, os.environ[variable]))

ALIAS: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/aliases.en.tsv.gz"
CLAIMS: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/claims.tsv.gz"
DESCRIPTION: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/descriptions.en.tsv.gz"
DWD_ISA: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/derived.dwd_isa.tsv.gz"
EXAMPLES_DIR: "/Users/pedroszekely/Documents/GitHub/kgtk-at-2021-wikidata-workshop"
EXTERNAL_ID: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/claims.external-id.tsv.gz"
ISA: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/derived.isa.tsv.gz"
ITEM: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/claims.wikibase-item.tsv.gz"
LABEL: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/labels.en.tsv.gz"
OUT: "/Users/pedroszekely/Downloads/kypher/wd-workshop"
P279: "/Volumes/GoogleDrive/Shar

In [7]:
%cd {output_path}

/Users/pedroszekely/Downloads/kypher


Define the shortcuts for Kypher so that import the relevant files into the Kypher index and define shortcuts to make the queries nicer to write

In [6]:
!$kypher \
-i "$ITEM" --as items \
-i "$TIME" --as time \
-i "$P31" --as p31 \
-i "$P279" --as p279 \
-i "$LABEL" --as labels \
-i "$P279STAR" --as p279star \
-i "$EXTERNAL_ID" --as external_ids \
-i "$OUT"/ulan.tsv --as ulan \
--limit 10

[2021-07-24 18:19:54 sqlstore]: IMPORT graph directly into table graph_1 from /Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/claims.wikibase-item.tsv.gz ...
[2021-07-24 19:12:25 sqlstore]: IMPORT graph directly into table graph_2 from /Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/claims.time.tsv.gz ...
[2021-07-24 19:18:12 sqlstore]: IMPORT graph directly into table graph_3 from /Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/derived.P31.tsv.gz ...
[2021-07-24 19:24:51 sqlstore]: IMPORT graph directly into table graph_4 from /Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/derived.P279.tsv.gz ...
[2021-07-24 19:25:20 sqlstore]: IMPORT graph directly into table graph_5 from /Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/labels.en.tsv.gz ...
[2021-07-24 19:34:51 sqlstore]: IMPORT graph directly into table graph_6 from /Volumes/GoogleDrive/Shared drives/KGTK/datasets/

## Retrieve large amounts of data from Wikidata

John is doing research on the popularity of first names to improve his entity resolution algorithm for people. He sees that Wikidata contains about 9 million people, so he wants to get the distribution of counts of first names from Wikidata. He writes a SPARQL query, but it times out, so he downloads the Wikidata KGTK files on his laptop and writes a kypher query. The query retrieves all instances of human (Q5), gets their frst names using the P735 property and return the counts.

John thinks he will want to do additional analysis on the data, so chooses standard KGTK names for the headers to generate the data as a KGTK graph that then he can use as input to other KGTK commands.

In [7]:
# compare to SPARQL
# common names
!$kypher -i items -i p31 -i labels \
--match '\
    p31: (person)-[]->(:Q5), \
    items: (person)-[:P735]->(given_name), \
    labels: (given_name)-[]->(given_name_label)' \
--return 'distinct given_name as node1, count(given_name) as node2, given_name_label as `node1;label`, "count_names" as label' \
--order-by 'node2 desc' \
-o "$OUT"/given-names.tsv

[2021-07-24 20:00:49 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_1_c2."node2" "_aLias.node1", count(graph_1_c2."node2") "_aLias.node2", graph_5_c3."node2" "_aLias.node1;label", ? "_aLias.label"
     FROM graph_1 AS graph_1_c2
     INNER JOIN graph_3 AS graph_3_c1, graph_5 AS graph_5_c3
     ON graph_1_c2."node2" = graph_5_c3."node1"
        AND graph_3_c1."node1" = graph_1_c2."node1"
        AND graph_1_c2."label" = ?
        AND graph_3_c1."node2" = ?
     GROUP BY "_aLias.node1"
     ORDER BY "_aLias.node2" DESC
  PARAS: ['count_names', 'P735', 'Q5']
---------------------------------------------
[2021-07-24 20:00:49 sqlstore]: CREATE INDEX on table graph_1 column node1 ...
[2021-07-24 20:07:24 sqlstore]: ANALYZE INDEX on table graph_1 column node1 ...
[2021-07-24 20:08:19 sqlstore]: CREATE INDEX on table graph_5 column node1 ...
[2021-07-24 20:09:13 sqlstore]: ANALYZE INDEX on table graph_5 column node1 ...
[2021-07-24 20:09:19 sqlst

In [8]:
!wc "$OUT"/given-names.tsv 

   53253  216696 1988754 /Users/pedroszekely/Downloads/kypher/wd-workshop/given-names.tsv


John takes a peek at the file to make sure he got the headers correcly: an edge from the q-node to the count, using `count_names` as the property, and including the `label` of `node1` so he can read the data. John sees that his name is by far the most popular name in Wikidata, and gets the information he needs to fine tune his entity resolution algorithms. 

In [9]:
!head "$OUT"/given-names.tsv | column -ts $'\t'

node1      node2   node1;label   label
Q4925477   120416  'John'@en     count_names
Q12344159  74235   'William'@en  count_names
Q4927937   59298   'Robert'@en   count_names
Q16428906  57107   'Thomas'@en   count_names
Q677191    52568   'James'@en    count_names
Q18057751  49005   'David'@en    count_names
Q2958359   44735   'Charles'@en  count_names
Q2793400   40987   'Peter'@en    count_names
Q1249148   40149   'Richard'@en  count_names


John gets curious and wants to know whether the popularity of names depends of time, so modifies his query to partition the data by people's year of birth.

In [10]:
!$kypher -i items -i time -i p31 -i labels \
--match '\
    p31: (person)-[]->(:Q5), \
    items: (person)-[:P735]->(given_name), \
    time: (person)-[:P569]->(date_of_birth), \
    labels: (given_name)-[]->(given_name_label)' \
--return 'distinct given_name as node1, kgtk_date_year(date_of_birth) as year, count(given_name) as node2, given_name_label as `node1;label`, "count_names_yearly" as label' \
--order-by 'given_name, cast(year, integer), node2 desc' \
-o "$OUT"/given-names.year.tsv

[2021-07-24 20:40:41 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_1_c2."node2" "_aLias.node1", kgtk_date_year(graph_2_c3."node2") "_aLias.year", count(graph_1_c2."node2") "_aLias.node2", graph_5_c4."node2" "_aLias.node1;label", ? "_aLias.label"
     FROM graph_1 AS graph_1_c2
     INNER JOIN graph_2 AS graph_2_c3, graph_3 AS graph_3_c1, graph_5 AS graph_5_c4
     ON graph_1_c2."node2" = graph_5_c4."node1"
        AND graph_3_c1."node1" = graph_1_c2."node1"
        AND graph_3_c1."node1" = graph_2_c3."node1"
        AND graph_1_c2."label" = ?
        AND graph_2_c3."label" = ?
        AND graph_3_c1."node2" = ?
     GROUP BY "_aLias.node1", "_aLias.year"
     ORDER BY graph_1_c2."node2" ASC, CAST("_aLias.year" AS integer) ASC, "_aLias.node2" DESC
  PARAS: ['count_names_yearly', 'P735', 'P569', 'Q5']
---------------------------------------------
[2021-07-24 20:40:41 sqlstore]: CREATE INDEX on table graph_2 column node1 ...
[2021-07-24 20:

In [11]:
!wc "$OUT"/given-names.year.tsv 

  882179 4431755 42410662 /Users/pedroszekely/Downloads/kypher/wd-workshop/given-names.year.tsv


John takea a quick peek at the file to verify that the headers are correct.

In [12]:
!head -5 "$OUT"/given-names.year.tsv | column -ts $'\t'

node1     year  node2  node1;label      label
Q1000387  1798  1      'Ferdinanda'@en  count_names_yearly
Q1000387  1849  1      'Ferdinanda'@en  count_names_yearly
Q1000387  1868  1      'Ferdinanda'@en  count_names_yearly
Q1000387  1870  1      'Ferdinanda'@en  count_names_yearly


John heard anecdotaly that Jessica had become a popular name in the late 90s and greps for Jessica in the file. 

In [13]:
!grep "'Jessica'" "$OUT"/given-names.year.tsv | tail -15

Q630846	1995	58	'Jessica'@en	count_names_yearly
Q630846	1996	41	'Jessica'@en	count_names_yearly
Q630846	1997	27	'Jessica'@en	count_names_yearly
Q630846	1998	23	'Jessica'@en	count_names_yearly
Q630846	1999	23	'Jessica'@en	count_names_yearly
Q630846	2000	51	'Jessica'@en	count_names_yearly
Q630846	2001	18	'Jessica'@en	count_names_yearly
Q630846	2002	18	'Jessica'@en	count_names_yearly
Q630846	2003	11	'Jessica'@en	count_names_yearly
Q630846	2004	6	'Jessica'@en	count_names_yearly
Q630846	2005	2	'Jessica'@en	count_names_yearly
Q630846	2009	1	'Jessica'@en	count_names_yearly
Q630846	2011	1	'Jessica'@en	count_names_yearly
Q630846	2014	1	'Jessica'@en	count_names_yearly
Q630846	2016	2	'Jessica'@en	count_names_yearly


John realizes that he needs to normalize the counts of names by the number of people born in each year. He wonders whether he can do it in one kypher query, but takes the easy way out and writes a simple query to get the counts of people born each year. He can do this faster than he can think of a complex query to get the final result in one go.

In [14]:
!$kypher -i time -i p31 \
--match ' \
    p31: (person)-[]->(:Q5), \
    time: (person)-[:P569]->(date_of_birth)' \
--return 'kgtk_date_year(date_of_birth) as node1, count(person) as node2, "count_people_born" as label' \
-o "$TEMP"/human.count.year.tsv

[2021-07-24 20:49:59 query]: SQL Translation:
---------------------------------------------
  SELECT kgtk_date_year(graph_2_c2."node2") "_aLias.node1", count(graph_3_c1."node1") "_aLias.node2", ? "_aLias.label"
     FROM graph_2 AS graph_2_c2
     INNER JOIN graph_3 AS graph_3_c1
     ON graph_3_c1."node1" = graph_2_c2."node1"
        AND graph_2_c2."label" = ?
        AND graph_3_c1."node2" = ?
     GROUP BY "_aLias.node1"
  PARAS: ['count_people_born', 'P569', 'Q5']
---------------------------------------------
       84.38 real        33.21 user        15.57 sys


John is happy that KGTK accepts literals as subjects of triples because here the subjects (`node1`) are years.

In [15]:
!head "$TEMP"/human.count.year.tsv

node1	node2	label
1	105	count_people_born
2	5	count_people_born
3	9	count_people_born
4	8	count_people_born
5	11	count_people_born
6	10	count_people_born
7	7	count_people_born
8	5	count_people_born
9	9	count_people_born


John knows he is almost there. He needs to get the names from the `given-names.year.tsv` file, and needs to pick out the year from the qualifier he put on the edge using the syntax to get the attributes of edges `[r {year: the_year}]`. He computes the fraction of people with each name and multiplies by 100,000 so that the numbers are not so tiny and easier to read. John also gets the labels of the q-nodes from the attribute he put on `node1` so tha the doesn't have to join with the `labels.tsv` file again.

In [16]:
!$kypher -i "$OUT"/given-names.year.tsv -i "$TEMP"/human.count.year.tsv \
--match ' \
    names: (given_name {label: given_name_label})-[r {year: the_year}]->(count_names), \
    year: (the_year)-[]->(count_people)' \
--return 'given_name as node1, "normalized_count_names_yearly" as label, cast(count_names, float) * 10000 / cast(count_people, float) as node2, the_year as year, given_name_label as `node1;label`' \
--order-by 'given_name, cast(the_year, integer), node2 desc' \
-o "$OUT"/given-names.year.normalized.tsv

[2021-07-24 20:51:24 sqlstore]: IMPORT graph directly into table graph_9 from /Users/pedroszekely/Downloads/kypher/wd-workshop/given-names.year.tsv ...
[2021-07-24 20:51:28 sqlstore]: IMPORT graph directly into table graph_10 from /Users/pedroszekely/Downloads/kypher/temp.wd-workshop/human.count.year.tsv ...
[2021-07-24 20:51:28 query]: SQL Translation:
---------------------------------------------
  SELECT graph_9_c1."node1" "_aLias.node1", ? "_aLias.label", (CAST(graph_9_c1."node2" AS float) * (? / CAST(graph_10_c2."node2" AS float))) "_aLias.node2", graph_10_c2."node1" "_aLias.year", graph_9_c1."node1;label" "_aLias.node1;label"
     FROM graph_10 AS graph_10_c2
     INNER JOIN graph_9 AS graph_9_c1
     ON graph_10_c2."node1" = graph_9_c1."year"
        AND graph_9_c1."node1;label" = graph_9_c1."node1;label"
        AND graph_9_c1."year" = graph_10_c2."node1"
     ORDER BY graph_9_c1."node1" ASC, CAST(graph_10_c2."node1" AS integer) ASC, "_aLias.node2" DESC
  PARAS: ['normalized_co

In [17]:
!head "$OUT"/given-names.year.normalized.tsv | column -ts $'\t'

node1     label                          node2               year  node1;label
Q1000387  normalized_count_names_yearly  2.207505518763797   1798  'Ferdinanda'@en
Q1000387  normalized_count_names_yearly  1.1767474699929394  1849  'Ferdinanda'@en
Q1000387  normalized_count_names_yearly  0.6827336655970506  1868  'Ferdinanda'@en
Q1000387  normalized_count_names_yearly  0.6334726973267453  1870  'Ferdinanda'@en
Q1000387  normalized_count_names_yearly  0.536711034778875   1888  'Ferdinanda'@en
Q1000433  normalized_count_names_yearly  1.0892059688487092  1852  'Bud'@en
Q1000433  normalized_count_names_yearly  0.9004141905276427  1858  'Bud'@en
Q1000433  normalized_count_names_yearly  0.6082355087890031  1881  'Bud'@en
Q1000433  normalized_count_names_yearly  0.5845218611176058  1882  'Bud'@en


John greps the normalized file again. Jessica was not a popular name in the 60s and began to get popular in the late 70s. John satisfied his curiosity. The popularity of names is time dependent, but for now, John will work to integrate the aggregate data into his entity resolution algorithm.

In [18]:
!grep "'Jessica'" "$OUT"/given-names.year.normalized.tsv | tail -50

Q630846	normalized_count_names_yearly	1.5782828282828283	1960	'Jessica'@en
Q630846	normalized_count_names_yearly	1.6427682992654478	1961	'Jessica'@en
Q630846	normalized_count_names_yearly	0.6936416184971098	1962	'Jessica'@en
Q630846	normalized_count_names_yearly	2.2975301550832854	1963	'Jessica'@en
Q630846	normalized_count_names_yearly	2.7612232218872963	1964	'Jessica'@en
Q630846	normalized_count_names_yearly	3.534651365553644	1965	'Jessica'@en
Q630846	normalized_count_names_yearly	3.7685601587820012	1966	'Jessica'@en
Q630846	normalized_count_names_yearly	2.2576760987357014	1967	'Jessica'@en
Q630846	normalized_count_names_yearly	3.7844383893430216	1968	'Jessica'@en
Q630846	normalized_count_names_yearly	4.271034846619601	1969	'Jessica'@en
Q630846	normalized_count_names_yearly	5.083022704168078	1970	'Jessica'@en
Q630846	normalized_count_names_yearly	8.008208413623965	1971	'Jessica'@en
Q630846	normalized_count_names_yearly	5.099569086412198	1972	'Jessica'@en
Q630846	normalized_count_names

## Analytics on full Wikidata

Jessica is working with John on the entity resolution algorithm and her job is to use the number of instances of each class in Wikidata as a feature. The query that Jessica needs to write is simple as she just needs to count the number of instances of each class, summing up over the instances of all subclasses. She knows that there are over 1 million classes in Wikidata (entities with a P279 property), so she knows it will not run on the public SPARQL endpint. Jessica gets the SQLite database from John so that she does not have to wait the 2 or so hours to load it on her laptop, writes the query and goes for lunch as she knows it will take a while for it to run.

In [19]:
# compare to SPARQL
!$kypher -i p31 -i p279star \
--match '\
    p31: (entity)-[]->(class), \
    p279star: (class)-[]->(super_class)' \
--return 'distinct super_class as node1, count(distinct entity) as node2, "entity_count" as label' \
--order-by 'node2 desc, node1' \
-o "$OUT"/class.count.tsv.gz

[2021-07-24 20:51:34 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_6_c2."node2" "_aLias.node1", count(DISTINCT graph_3_c1."node1") "_aLias.node2", ? "_aLias.label"
     FROM graph_3 AS graph_3_c1
     INNER JOIN graph_6 AS graph_6_c2
     ON graph_3_c1."node2" = graph_6_c2."node1"
     GROUP BY "_aLias.node1"
     ORDER BY "_aLias.node2" DESC, "_aLias.node1" ASC
  PARAS: ['entity_count']
---------------------------------------------
[2021-07-24 20:51:34 sqlstore]: CREATE INDEX on table graph_6 column node1 ...
[2021-07-24 20:52:27 sqlstore]: ANALYZE INDEX on table graph_6 column node1 ...
^C


After coming back from lunch, the file is ready, it contains data for 75K classes, she figures that the other classes don't have instances.

In [20]:
!zcat < "$OUT"/class.count.tsv.gz | wc

zcat: (stdin): unexpected end of file
       0       0       0


In [21]:
!zcat < "$OUT"/class.count.tsv.gz | head -5 

zcat: (stdin): unexpected end of file


Jessica is curious about the data, so she writes a query to get the counts of different classes of film (Q11424). Jessica had been working with John, so she learned the trick to use the standard names for column headings so that she can use the output of previous queries as new graphs. She shudders to think that if she was using SPARQL she would have had to set up a new Wikidata SPARQL endpoint to be able to load her personal data in it, and to be extremely caeful to not make a mistake because deleting the data would have been a chore. Jessica had watched John make several mistakes when he was building the files for the names. John had simply fixed the queries and re-run the other queries that depended on the data he had just fixed.

In [22]:
!$kypher -i p279star -i labels -i "$OUT"/class.count.tsv.gz \
--match ' \
    p279star: (class)-[]->(:Q11424), \
    count: (class)-[]->(count), \
    labels: (class)-[]->(class_label)' \
--return 'class as class, class_label as name, count as count' \
--order-by 'cast(count, integer) desc' \
--limit 10 \
| column -ts $'\t'

^C
Traceback (most recent call last):
  File "/Users/pedroszekely/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/pkg_resources/__init__.py", line 568, in _build_master
    ws.require(__requires__)
  File "/Users/pedroszekely/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/pkg_resources/__init__.py", line 886, in require
    needed = self.resolve(parse_requirements(requirements))
  File "/Users/pedroszekely/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/pkg_resources/__init__.py", line 777, in resolve
    raise VersionConflict(dist, req).with_context(dependent_req)
pkg_resources.ContextualVersionConflict: (etk 2.2.3a0 (/Users/pedroszekely/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages), Requirement.parse('etk>=2.2.6'), {'kgtk'})

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/pedroszekely/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/pkg_resources/__init__.py", line 3016,

Jessica now has the statistics she needs to work on her feature for the entity resolution algorithm. 

## Extract new graphs from Wikidata

Bill is working on a project to find networks of researchers working on specific topics. He wants to use publication data to find relationships among authors using publications. Bill knows that he can get lots of publication data from Pubmed or Microsoft Academic graph, but wants to give Wikidata a try as he heard that Wikidata has close to 40 million publications, and that in Wikidata publications have links to other entities such as main subjects.

Bill decides that the simplest experiment to try first is to build a network of authors of publications in Wikidata: he wants to create a graph of people in Wikidata who authored papers, to put a link between two people if the coauthored a paper, and to add a qualifier with the count of papers they coauthored. He knows the computation is expensive as there are 40ish million papers in Wikidata, so the network will be large. He doesn't even try to write a SPARQL query because he knows it will time out. Bill downloads the KGTK files and decides to write his first query using only 2019 data so he doesn't have to wait so long if he makes a mistake.

### First do it for 2019 to debug the query

In [ ]:
!$kypher -i p31 -i p279star -i items -i time -i labels \
--match '\
    p31: (pub)-[]->(class), \
    p279star: (class)-[]->(:Q591041), \
    time: (pub)-[:P577]->(pub_date), \
    items: (pub)-[:P50]->(author1), \
    items: (pub)-[:P50]->(author2), \
    labels: (author1)-[]->(author1_label)' \
--where 'author1 != author2 and kgtk_date_year(pub_date) = 2019' \
--return 'distinct author1 as node1, "Pcoauthor" as relation, author2 as node2, count(distinct pub) as count_publications, author1_label as `node1;label`' \
--order-by 'count_publications desc' \
-o "$TEMP"/coauthors.2019.id.tsv.gz

In [ ]:
!zcat < "$TEMP"/coauthors.2019.id.tsv.gz | head

Bill wants to sanity check his data so he looks up the first person in Google Scholar and finds that Secundino López Puente has many publications in 2019. Looks like the query is working fine.

### Build the network for all authors

Bill removes the year restriction and runs the query for the full data. The query for a single year took close to 10 minutes, so Bill decides to leave the query running overnight.

In [ ]:
# compare to SPARQL
!$kypher -i p31 -i p279star -i items -i time -i labels \
--match '\
    p31: (pub)-[]->(class), \
    p279star: (class)-[]->(:Q591041), \
    time: (pub)-[:P577]->(pub_date), \
    items: (pub)-[:P50]->(author1), \
    items: (pub)-[:P50]->(author2)' \
--where 'author1 != author2' \
--return 'distinct author1 as node1, "Pcoauthor" as relation, author2 as node2, count(distinct pub) as count_publications' \
--order-by 'count_publications desc' \
-o "$TEMP"/coauthors.2019.tsv.gz

### Build a network of authors who authored papers about cancer

Bill is interested in cancer research, so he wants to build the same network but using only the papers about cancer. He knows Wikidata has an extensive class hiearchy, so he writes a query to peek at the hierarchy below the q-node for cancer.
He writes a query to retrieve subclasses of cancer.

In [ ]:
!$kypher -i p279star -i labels \
--match '\
    p279star: (cancer_type)-[]->(:Q12078), \
    labels: (cancer_type)-[]->(cancer_type_label)' \
--return 'cancer_type as node1, cancer_type_label as node2' \
--limit 10

The results are promising, so Bill now incorporates the query for types of cancer into the query for building the coauthor network. He just needs to get the main subject of the paper using the `P921` property and test that the main subject is a subclass of cancer. He expects the query to be much faster because now it has strong restriction, so he gives it a try.

In [ ]:
# compare to SPARQL
!$kypher -i p31 -i p279star -i items -i labels \
--match '\
    p31: (pub)-[]->(class), \
    p279star: (class)-[]->(:Q591041), \
    items: (pub)-[:P50]->(author1), \
    items: (pub)-[:P50]->(author2), \
    items: (pub)-[:P921]->(cancer_type), \
    p279star: (cancer_type)-[]->(:Q12078), \
    labels: (author1)-[]->(author1_label), \
    labels: (author2)-[]->(author2_label)' \
--where 'author1 != author2' \
--return 'distinct author1 as node_x, "Pcoauthor" as relation, author2 as node_y, count(distinct pub) as count_publications, author1_label as `node1;label`, author2_label as `node2;label`' \
--order-by 'count_publications desc' \
-o "$TEMP"/coauthors.cancer.tsv.gz

The query takes less than a minute and produces a network with close to half a million edges. Bill takes a peek to see what is in it, and now wonders whether he could have written the query in SPARQL and run it on the public SPARQL endpoint.

In [ ]:
!zcat < "$TEMP"/coauthors.cancer.tsv.gz | wc

In [ ]:
!zcat < "$TEMP"/coauthors.cancer.tsv.gz | head | column -ts $'\t'

Bill puts the first two names in Google and finds that they are famous and have publshied a lot together. Bill is happy to have a network with close to half a million edges that he can use to do interesting analyses.

## Combining Wikidata with external files

Abigail is working on a cultural heritage project, collaborating with the Getty Research Institute who gave her a file with 27 thousand ULAN identifiers. Abigail has a database indexed using VIAF identifiers, and wants to map her ULAN identifiers to VIAF identifiers so that she can use her database. She puts one of the ULAN identifiers in the Wikidata search box and discovers that Wikidata has both ULAN and VIAF identifiers for many artists. Abigail knows a little bit of SPARQL and easity figures out that it is easy to write a query to retrieve the VIAF identifier given a ULAN identifier. Her solution would require sending 27,000 queries to Wikidata, which would involve writing a Python script.

In [ ]:
!wc "$OUT"/ulan.tsv

Her colleague Bill tells her that she can easily solve the problem using KGTK query. The only thing she needs to do is to rename the header of her file with identifiers to `node1` and write a Kypher query.

In [ ]:
# compare to SPARQL
!$kypher -i items -i external_ids -i labels -i ulan \
--match '\
    ulan: (ulan_id)-[]->(), \
    external_ids: (viaf_id)<-[:P214]-(artist)-[:P245]->(ulan_id), \
    labels: (artist)-[]->(artist_label)' \
--return 'artist as qnode, viaf_id as viaf, ulan_id as ulan, artist_label as name' \
-o "$OUT"/ulan-to-viaf.tsv

Abigail is thrilled to see that the query ran in less than 30 seconds and is curious to see the results. She got matches for 8,116 ULAN ids, which means that now she can get a lot of data from her database to do her analysis.

In [ ]:
!wc "$OUT"/ulan-to-viaf.tsv

In [ ]:
!head "$OUT"/ulan-to-viaf.tsv | column -ts $'\t'

## Combining Wikidata with DBpedia

After mining her VIAF database, Abigail realizes that there is a lot of interesting data in Wikipedia infoboxes that she would like to use in her analysis. She hears from a colleague that DBpedia extracts data from Wikipedia infoboxes. She is curious whether Wikidata already has most of this data. She browses the pages for some artists in Wikipedia and sees that the Wikipedia infoboxes have interesting information that she may want to include in her dataset.

Abigail downloads the DBpedia infobox data in RDF format and uses KGTK to convert it into KGTK format and to substitute the DBpedia URIs with Wikidata Q-nodes. 

In [8]:
!zcat < "$OUT"/wikidata_infobox.tsv.gz | wc -l

 99251608


Abigail sees that there are almost 100 million edges in the DBpedia infobox graph, so she first adds the dataset to the kypher index.
> This operation is similar to loading the triples into a triple store to enable running queries. In kypher it is not necessary to explicitly load the file as kypher will automatically load the file and build indices the first time the file is used. Abigail is doing separately as she is curious to see how long it takes to load the file in kypher.

In [11]:
!$kypher -i "$OUT"/wikidata_infobox.tsv.gz --as infobox \
--limit 1

[2021-07-24 21:20:28 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_11 AS graph_11_c1
     LIMIT ?
  PARAS: [1]
---------------------------------------------
node1	label	node2	id
nodemxZbyK2VRrGoaxfdLmyLxw-1	dbpedia:structured_value	"2019-08-07"	nodemxZbyK2VRrGoaxfdLmyLxw-1-dbpedia:structured_value-944f0e
        2.06 real         0.85 user         0.19 sys


Abigail first wants to see which properties are available for the people she has in the ULAN file. She constructs a query to count the number of statements for each property for the artists in her ULAN file.

> This query combines Wikidata with two external sources, her ULAN identifiers and DBPedia infoboxes.

In [ ]:
!$kypher -i infobox -i ulan -i external_ids \
--match '\
    ulan: (ulan_id)-[]->(), \
    external_ids: (artist)-[:P245]->(ulan_id), \
    infobox: (artist)-[l]->()' \
--return 'l.label as node1, count(distinct l) as node2' \
--order-by 'node2 desc' \
--limit 20 \
| column -ts $'\t'

Abigail is interested in the information about spouses as she is thinking of doing an analysis on the occupation of spouses. She constructs a query to retrieve the spouse statements already present in Wikidata for her ULAN artists.

In [ ]:
!$kypher -i items -i ulan -i external_ids -i labels \
--match '\
    ulan: (ulan_id)-[]->(), \
    external_ids: (artist)-[:P245]->(ulan_id), \
    items: (artist)-[l:P26]->(spouse)' \
--return 'artist as node1, l.label as label, spouse as node2' \
-o "$OUT"/spouses.ulan.wikidata.tsv

In [ ]:
!wc "$OUT"/spouses.ulan.wikidata.tsv

She also constructs a query to count the spouse statements of ULAN artists in the DBpedia dataset an converts the DBpedia property to `P26`, the Wikidata property.

In [ ]:
!$kypher -i items -i infobox -i ulan -i external_ids -i labels \
--match '\
    ulan: (ulan_id)-[]->(), \
    external_ids: (artist)-[:P245]->(ulan_id), \
    infobox: (artist)-[:`property:spouse`]->(spouse)' \
--return 'artist as node1, "P26" as label, spouse as node2' \
-o "$OUT"/spouses.ulan.dbpedia.tsv

Abigail is encouraged as there are almost 1,000 additional statements in DBpedia that she may be able to import into her dataset.

In [ ]:
!wc "$OUT"/spouses.ulan.dbpedia.tsv

Abigail worries that some of the spouses may be strings rather than entities as she has seen Wikipedia infoboxes where some values are links and others are strings. She uses the regex feature in Kypher to count the number of spouses that are Wikidata q-nodes. 

In [ ]:
!$kypher -i "$OUT"/spouses.ulan.dbpedia.tsv \
--match '()-[]->(spouse)' \
--where 'spouse =~ "^Q[0-9]+"' \
--return 'count(distinct spouse) as count_spouses_with_qnodes'

Abigail is disappointed to see that only 449 are q-nodes, so uses grep to see what else is in the file. She sees that the DBpedia data is noisy as there are empty strings, numbers that look like dates and entities that do not correspond to Wikidata entities. 

In [ ]:
!grep -v '\tQ[0-9]' "$OUT"/spouses.ulan.dbpedia.tsv | head

Abigail does not want to take on what looks like a difficult data cleaning and entity linking job, so she keeps the 449 clean entities and puts them in `spouses.ulan.dbpedia.qnodes.tsv`.

In [ ]:
!grep '\tQ[0-9]' "$OUT"/spouses.ulan.dbpedia.tsv > "$OUT"/spouses.ulan.dbpedia.qnodes.tsv

Abigail wants to see how many artists have spouse statements in both Wikidata and DBpedia, so she adds another clause to the query and sees that only 359 are in both, so she can get about 100 new statements from DBpedia. While not a lot, she will diff the two files and keep the new statements (not shown in this notebook).

In [22]:
!$kypher -i items -i infobox -i ulan -i external_ids -i labels \
--match '\
    ulan: (ulan_id)-[]->(), \
    external_ids: (artist)-[:P245]->(ulan_id), \
    infobox: (artist)-[l:`property:spouse`]->(spouse), \
    items: (artist)-[:P26]->(spouse), \
    labels: (spouse)-[]->(spouse_label)' \
--return 'count(distinct l)' 

[2021-07-24 22:33:33 query]: SQL Translation:
---------------------------------------------
  SELECT count(DISTINCT graph_11_c3."id")
     FROM graph_1 AS graph_1_c4
     INNER JOIN graph_11 AS graph_11_c3, graph_5 AS graph_5_c5, graph_7 AS graph_7_c2, graph_8 AS graph_8_c1
     ON graph_11_c3."node2" = graph_1_c4."node2"
        AND graph_11_c3."node2" = graph_5_c5."node1"
        AND graph_7_c2."node1" = graph_11_c3."node1"
        AND graph_7_c2."node1" = graph_1_c4."node1"
        AND graph_8_c1."node1" = graph_7_c2."node2"
        AND graph_11_c3."label" = ?
        AND graph_1_c4."label" = ?
        AND graph_7_c2."label" = ?
  PARAS: ['property:spouse', 'P26', 'P245']
---------------------------------------------
[2021-07-24 22:33:33 sqlstore]: CREATE INDEX on table graph_7 column node2 ...
[2021-07-24 22:37:57 sqlstore]: ANALYZE INDEX on table graph_7 column node2 ...
[2021-07-24 22:38:10 sqlstore]: CREATE INDEX on table graph_7 column label ...
[2021-07-24 22:40:53 sqlstore]: 

### spouse statements in DBpedia
Abigail gets curious wether it is worth working on a project to augment Wikidata with spouse statements from DBpedia. Now she knows that she needs to focus on the ones that are mapped to q-nodes, so writes a query to fetch all the spouse statements from DBpedia. She wants to make sure to get clean data, so she adds a constraint to verify that the q-nodes she gets from DBpedia are instances of `Q5`  (human) in Wikidata.

In [13]:
# dbpedia spouses
!$kypher -i infobox -i p31 \
--match ' \
    infobox: (artist)-[:`property:spouse`]->(spouse), \
    p31: (spouse)-[]->(:Q5)' \
--return 'artist as node1, "P26" as label, spouse as node2' \
-o "$OUT"/spouses.dbpedia.qnodes.tsv

[2021-07-24 21:21:06 query]: SQL Translation:
---------------------------------------------
  SELECT graph_11_c1."node1" "_aLias.node1", ? "_aLias.label", graph_11_c1."node2" "_aLias.node2"
     FROM graph_11 AS graph_11_c1
     INNER JOIN graph_3 AS graph_3_c2
     ON graph_11_c1."node2" = graph_3_c2."node1"
        AND graph_11_c1."label" = ?
        AND graph_3_c2."node2" = ?
  PARAS: ['P26', 'property:spouse', 'Q5']
---------------------------------------------
[2021-07-24 21:21:06 sqlstore]: CREATE INDEX on table graph_11 column label ...
[2021-07-24 21:23:08 sqlstore]: ANALYZE INDEX on table graph_11 column label ...
[2021-07-24 21:23:15 sqlstore]: CREATE INDEX on table graph_11 column node2 ...
[2021-07-24 21:25:15 sqlstore]: ANALYZE INDEX on table graph_11 column node2 ...
      323.33 real       219.21 user        41.19 sys


In [14]:
!wc -l "$OUT"/spouses.dbpedia.qnodes.tsv

   39778 /Users/pedroszekely/Downloads/kypher/wd-workshop/spouses.dbpedia.qnodes.tsv


Abigail also gets all the spouse statements from Wikidata

In [15]:
!$kypher -i items \
--match '(artist)-[l:P26]->(spouse)' \
--return 'artist as node1, l.label as label, spouse as node2' \
-o "$OUT"/spouses.wikidata.tsv

[2021-07-24 21:30:04 query]: SQL Translation:
---------------------------------------------
  SELECT graph_1_c1."node1" "_aLias.node1", graph_1_c1."label" "_aLias.label", graph_1_c1."node2" "_aLias.node2"
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label" = ?
  PARAS: ['P26']
---------------------------------------------
       47.16 real         3.92 user         6.35 sys


In [16]:
!wc -l "$OUT"/spouses.wikidata.tsv 

  681174 /Users/pedroszekely/Downloads/kypher/wd-workshop/spouses.wikidata.tsv


Abigail sees that there is a significant difference, so writes a query to identify the common statements in both datasets and sees that there are about clean 7,000 spouse statements in DBpedia that are not present in Wikidata. It will be easy to add them using Wikidata quick statements, a project that she will try to do later.
> Kypher can run queries over the two new graphs created in the previous queries

In [17]:
!$kypher -i "$OUT"/spouses.wikidata.tsv  -i "$OUT"/spouses.dbpedia.qnodes.tsv \
--match '\
    wikidata: (person)-[]->(spouse), \
    dbpedia: (person)-[]->(spouse)' \
--return 'distinct person as person, spouse as spouse' \
| wc -l

[2021-07-24 21:30:51 sqlstore]: IMPORT graph directly into table graph_12 from /Users/pedroszekely/Downloads/kypher/wd-workshop/spouses.wikidata.tsv ...
[2021-07-24 21:30:53 sqlstore]: IMPORT graph directly into table graph_13 from /Users/pedroszekely/Downloads/kypher/wd-workshop/spouses.dbpedia.qnodes.tsv ...
[2021-07-24 21:30:53 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_12_c1."node1" "_aLias.person", graph_12_c1."node2" "_aLias.spouse"
     FROM graph_12 AS graph_12_c1
     INNER JOIN graph_13 AS graph_13_c2
     ON graph_12_c1."node1" = graph_13_c2."node1"
        AND graph_12_c1."node2" = graph_13_c2."node2"
  PARAS: []
---------------------------------------------
[2021-07-24 21:30:53 sqlstore]: CREATE INDEX on table graph_12 column node2 ...
[2021-07-24 21:30:53 sqlstore]: ANALYZE INDEX on table graph_12 column node2 ...
[2021-07-24 21:30:53 sqlstore]: CREATE INDEX on table graph_13 column node1 ...
[2021-07-24 21:30:53 sqlstor

## Validate property constraints in Wikidata

Amir and Sarah are starting a project to find constraint violations in Wikidata. They find that constraints are associated with properties using the `P2302` (property constraint) property. There are over 44,000 constraints, so finding violations is a dauting task as many constraints apply to a very large number of statements.

In [21]:
!$kypher -i items \
--match '(property)-[l:P2302]->(constraint)' \
--return 'count(distinct l) as count' 

[2021-07-24 21:44:29 query]: SQL Translation:
---------------------------------------------
  SELECT count(DISTINCT graph_1_c1."id") "_aLias.count"
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label" = ?
  PARAS: ['P2302']
---------------------------------------------
count
44552
        1.43 real         0.89 user         0.25 sys


Sarah refines the query to print the counts of the different types of constraints, and they see that 9 constraint types are significantly more popular than the others. Amir and Sarah decide to focus on the `value type constraint` as this is the constraint that checks that the value of a statement belongs to specific classes. This constraint is defined for 964 properties, so it is worth working on it.

In [20]:
!$kypher -i items -i labels -i p279star \
--match ' \
    items: (property)-[l:P2302]->(constraint), \
    labels: (constraint)-[]->(constraint_label)' \
--return 'constraint_label as constraint_label, count(distinct l) as count' \
--order-by 'count desc' \
| column -ts $'\t'

[2021-07-24 21:43:25 query]: SQL Translation:
---------------------------------------------
  SELECT graph_5_c2."node2" "_aLias.constraint_label", count(DISTINCT graph_1_c1."id") "_aLias.count"
     FROM graph_1 AS graph_1_c1
     INNER JOIN graph_5 AS graph_5_c2
     ON graph_1_c1."node2" = graph_5_c2."node1"
        AND graph_1_c1."label" = ?
     GROUP BY "_aLias.constraint_label"
     ORDER BY "_aLias.count" DESC
  PARAS: ['P2302']
---------------------------------------------
        1.34 real         0.91 user         0.23 sys
constraint_label                                        count
'item requires statement constraint'@en                 7576
'allowed entity types constraint'@en                    6401
'format constraint'@en                                  6042
'distinct values constraint'@en                         5513
'single value constraint'@en                            5453
'type constraint'@en                                    5070
'property scope constraint'@en   

In [23]:
!$kypher --show-cache

Graph Cache:
DB file: /Users/pedroszekely/Downloads/kypher/temp.novartis/wikidata.sqlite3.db
  size:  140.60 GB   	free:  0 Bytes   	modified:  2021-07-24 22:44:32

KGTK File Information:
/Users/pedroszekely/Downloads/kypher/temp.wd-workshop/human.count.year.tsv:
  size:  50.79 KB   	modified:  2021-07-24 20:51:20   	graph:  graph_10
/Users/pedroszekely/Downloads/kypher/wd-workshop/given-names.year.tsv:
  size:  40.45 MB   	modified:  2021-07-24 20:49:54   	graph:  graph_9
/Users/pedroszekely/Downloads/kypher/wd-workshop/spouses.dbpedia.qnodes.tsv:
  size:  826.70 KB   	modified:  2021-07-24 21:26:27   	graph:  graph_13
/Users/pedroszekely/Downloads/kypher/wd-workshop/spouses.wikidata.tsv:
  size:  15.26 MB   	modified:  2021-07-24 21:30:49   	graph:  graph_12
external_ids:
  size:  3.59 GB   	modified:  2021-03-11 06:19:15   	graph:  graph_7
infobox:
  size:  1.46 GB   	modified:  2021-07-24 11:42:08   	graph:  graph_11
items:
  size:  7.47 GB   	modified:  2021-02-26 08:03:09   	grap

In [ ]:
"Took {:.2f} minutes to run the notebook from start to end".format((round(time.time()) - start_time)/60)